In [2]:
import pandas as pd

In [3]:
import json

with open('./data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)  # agora `data` é um dicionário Python


In [4]:
df = pd.json_normalize(data["competitions"])


In [5]:
df.head(10)

,id,name,code,type,emblem,plan,numberOfAvailableSeasons,lastUpdated,area.id,area.name,...,currentSeason.winner.shortName,currentSeason.winner.tla,currentSeason.winner.crest,currentSeason.winner.address,currentSeason.winner.website,currentSeason.winner.founded,currentSeason.winner.clubColors,currentSeason.winner.venue,currentSeason.winner.lastUpdated,currentSeason
0,2166,AFC Champions League,ACL,CUP,None,TIER_FOUR,1,2023-02-26T12:57:33Z,2001,Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006,WC Qualification CAF,QCAF,CUP,None,TIER_FOUR,3,2022-03-13T18:51:44Z,2001,Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2176,Africa Cup,AC,CUP,https://crests.football-data.org/ac.png,TIER_FOUR,2,2024-01-13T11:34:43Z,2001,Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,Primera B Nacional,PBN,LEAGUE,None,TIER_FOUR,5,2023-08-01T08:40:24Z,2011,Argentina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,Liga Profesional,ASL,LEAGUE,https://crests.football-data.org/LPDF.svg,TIER_TWO,9,2024-09-13T16:34:03Z,2011,Argentina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2149,Copa Liga Profesional,None,LEAGUE,None,TIER_FOUR,2,2024-06-20T18:58:13Z,2011,Argentina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025,Supercopa Argentina,None,SUPER_CUP,None,TIER_FOUR,2,2019-05-03T05:08:18Z,2011,Argentina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2147,WC Qualification AFC,QAFC,CUP,None,TIER_FOUR,3,2021-11-11T10:31:11Z,2014,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2008,A League,AAL,LEAGUE,https://crests.football-data.org/a-league.png,TIER_TWO,8,2024-09-13T16:55:10Z,2015,Australia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2026,FFA Cup,None,CUP,None,TIER_FOUR,3,2019-10-23T23:59:22Z,2015,Australia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Extrair área
df_area = df[['area.id', 'area.name', 'area.code']].drop_duplicates().rename(
    columns={'area.id': 'id', 'area.name': 'name', 'area.code': 'code'}
)

# Extrair vencedores (alguns podem ser NaN, dropar esses)
df_winners = df[['currentSeason.winner.id', 'currentSeason.winner.name', 'currentSeason.winner.shortName', 'currentSeason.winner.tla',
                 'currentSeason.winner.crest', 'currentSeason.winner.address', 'currentSeason.winner.website',
                 'currentSeason.winner.founded', 'currentSeason.winner.clubColors', 'currentSeason.winner.venue',
                 'currentSeason.winner.lastUpdated']].dropna(subset=['currentSeason.winner.id']).drop_duplicates().rename(
    columns=lambda x: x.replace('currentSeason.winner.', '')
)

# Extrair temporadas
df_seasons = df[['currentSeason.id', 'currentSeason.startDate', 'currentSeason.endDate', 'currentSeason.currentMatchday', 'currentSeason.winner.id']].rename(
    columns={
        'currentSeason.id': 'id',
        'currentSeason.startDate': 'start_date',
        'currentSeason.endDate': 'end_date',
        'currentSeason.currentMatchday': 'current_matchday',
        'currentSeason.winner.id': 'winner_id'
    }
).drop_duplicates()

# Competitions (remover colunas aninhadas já extraídas)
df_competitions = df.drop(columns=[
    'area.id', 'area.name', 'area.code',
    'currentSeason.id', 'currentSeason.startDate', 'currentSeason.endDate', 'currentSeason.currentMatchday', 'currentSeason.winner.id',
    'currentSeason.winner.name', 'currentSeason.winner.shortName', 'currentSeason.winner.tla', 'currentSeason.winner.crest',
    'currentSeason.winner.address', 'currentSeason.winner.website', 'currentSeason.winner.founded', 'currentSeason.winner.clubColors',
    'currentSeason.winner.venue', 'currentSeason.winner.lastUpdated'
])

# Agora, adicionar foreign keys:
df_competitions['area_id'] = df['area.id']
df_competitions['current_season_id'] = df['currentSeason.id']

In [7]:
df_area.head(10)

,id,name,code
0,2001,Africa,AFR
3,2011,Argentina,ARG
7,2014,Asia,ASI
8,2015,Australia,AUS
10,2016,Austria,AUT
14,2023,Belgium,BEL
19,2028,Bolivia,BOL
20,2030,Bosnia and Herzegovina,BIH
21,2032,Brazil,BRA
26,2035,Bulgaria,BGR


In [8]:
df_area.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 0 to 177
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      61 non-null     int64 
 1   name    61 non-null     object
 2   code    61 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.9+ KB


In [9]:
df_winners.head(10)

,id,name,shortName,tla,crest,address,website,founded,clubColors,venue,lastUpdated
13,2022.0,spusu SKN St. Pölten,St. Pölten,STP,https://crests.football-data.org/2022.png,Bimbo Binder Promenade 9 Sankt Pölten 3100,http://www.skn-stpoelten.at,2000.0,Yellow / Blue,NV Arena,2023-08-23T08:41:01Z
54,760.0,Spain,Spain,ESP,https://crests.football-data.org/760.svg,"Ramón y Cajal, s/n Las Rozas 28230",http://www.rfef.es,1909.0,Red / Blue / Yellow,Estadio Alfredo Di Stéfano,2021-05-26T09:46:48Z
78,7522.0,VfL Wolfsburg,None,None,None,null null null,None,NaN,None,None,2018-10-03T20:17:40Z
181,6677.0,Spain,Spain,ESP,https://crests.football-data.org/760.svg,"Ramón y Cajal, s/n Las Rozas 28230",http://www.rfef.es,1909.0,Red / Blue / Yellow,Estadio Nuevo Municipal Las Gaunas,2023-04-05T11:14:25Z


In [10]:
df_winners.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 13 to 181
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           4 non-null      float64
 1   name         4 non-null      object 
 2   shortName    3 non-null      object 
 3   tla          3 non-null      object 
 4   crest        3 non-null      object 
 5   address      4 non-null      object 
 6   website      3 non-null      object 
 7   founded      3 non-null      float64
 8   clubColors   3 non-null      object 
 9   venue        3 non-null      object 
 10  lastUpdated  4 non-null      object 
dtypes: float64(2), object(9)
memory usage: 384.0+ bytes


In [11]:
df_competitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        183 non-null    int64  
 1   name                      183 non-null    object 
 2   code                      118 non-null    object 
 3   type                      183 non-null    object 
 4   emblem                    51 non-null     object 
 5   plan                      182 non-null    object 
 6   numberOfAvailableSeasons  183 non-null    int64  
 7   lastUpdated               183 non-null    object 
 8   area.flag                 151 non-null    object 
 9   currentSeason.winner      0 non-null      float64
 10  currentSeason             0 non-null      float64
 11  area_id                   183 non-null    int64  
 12  current_season_id         182 non-null    float64
dtypes: float64(3), int64(3), object(7)
memory usage: 18.7+ KB


In [14]:
df_competitions.head(10)

,id,name,code,type,emblem,plan,numberOfAvailableSeasons,lastUpdated,area.flag,currentSeason.winner,currentSeason,area_id,current_season_id
0,2166,AFC Champions League,ACL,CUP,None,TIER_FOUR,1,2023-02-26T12:57:33Z,None,NaN,NaN,2001,1559.0
1,2006,WC Qualification CAF,QCAF,CUP,None,TIER_FOUR,3,2022-03-13T18:51:44Z,None,NaN,NaN,2001,1609.0
2,2176,Africa Cup,AC,CUP,https://crests.football-data.org/ac.png,TIER_FOUR,2,2024-01-13T11:34:43Z,None,NaN,NaN,2001,2385.0
3,2023,Primera B Nacional,PBN,LEAGUE,None,TIER_FOUR,5,2023-08-01T08:40:24Z,https://crests.football-data.org/ar.svg,NaN,NaN,2011,716.0
4,2024,Liga Profesional,ASL,LEAGUE,https://crests.football-data.org/LPDF.svg,TIER_TWO,9,2024-09-13T16:34:03Z,https://crests.football-data.org/ar.svg,NaN,NaN,2011,2394.0
5,2149,Copa Liga Profesional,None,LEAGUE,None,TIER_FOUR,2,2024-06-20T18:58:13Z,https://crests.football-data.org/ar.svg,NaN,NaN,2011,717.0
6,2025,Supercopa Argentina,None,SUPER_CUP,None,TIER_FOUR,2,2019-05-03T05:08:18Z,https://crests.football-data.org/ar.svg,NaN,NaN,2011,430.0
7,2147,WC Qualification AFC,QAFC,CUP,None,TIER_FOUR,3,2021-11-11T10:31:11Z,None,NaN,NaN,2014,1616.0
8,2008,A League,AAL,LEAGUE,https://crests.football-data.org/a-league.png,TIER_TWO,8,2024-09-13T16:55:10Z,None,NaN,NaN,2015,2349.0
9,2026,FFA Cup,None,CUP,None,TIER_FOUR,3,2019-10-23T23:59:22Z,None,NaN,NaN,2015,592.0


In [15]:
df_seasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                182 non-null    float64
 1   start_date        182 non-null    object 
 2   end_date          182 non-null    object 
 3   current_matchday  139 non-null    float64
 4   winner_id         4 non-null      float64
dtypes: float64(3), object(2)
memory usage: 7.3+ KB


In [16]:
df_seasons.head(10)

,id,start_date,end_date,current_matchday,winner_id
0,1559.0,2022-04-06,2023-05-05,NaN,NaN
1,1609.0,2023-11-15,2025-10-13,7.0,NaN
2,2385.0,2025-12-21,2026-01-18,1.0,NaN
3,716.0,2021-03-13,2021-12-26,17.0,NaN
4,2394.0,2025-07-13,2025-11-16,3.0,NaN
5,717.0,2021-02-12,2021-05-30,13.0,NaN
6,430.0,2019-04-04,2019-04-04,NaN,NaN
7,1616.0,2023-10-12,2024-06-11,6.0,NaN
8,2349.0,2024-10-18,2025-05-04,29.0,NaN
9,592.0,2020-02-19,2020-10-28,NaN,NaN
